# doc2vec

Doc import

In [39]:
import pandas as pd
import time 

hotelData = pd.read_csv('../data/preprocessed/tokenTrue_remStpwrdsTrue_stemmTrue_lemmatizeFalse_nGramFalse_nGram_length2.csv')  
review = hotelData["Review"].apply(lambda row: row.strip("']['").split("', '"))
score = hotelData["Reviewer_Score"]

create wordlist of all inputs togehter

In [29]:
# Source: https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings/comments
from tqdm import tqdm
tqdm.pandas()

def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

C:\Users\Admin\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [30]:
vocab = build_vocab(review)

100%|██████████████████████████████████████████████████████████████████████| 515738/515738 [00:03<00:00, 164072.66it/s]


Load the model and the keyedVector

In [31]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
import gensim.models as g

firstTime = time.time()
#documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(abstractsPro)]
modelLNK = "C:\\Users\\Admin\\Downloads\\enwiki_dbow\\doc2vec.bin"


#inference hyper-parameters
start_alpha=0.01
infer_epoch=1000
 
#load model

model = g.Doc2Vec.load(modelLNK)
#model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

#embeddings_index = KeyedVectors.load_word2vec_format(modelLNK, binary=True)
embeddings_index = model.wv


# # Save file and load it again
# fname = get_tmpfile("my_doc2vec_model")

# model.save(fname)
# model = Doc2Vec.load(fname)  # you can continue training with the loaded model!
# #delete traingmode (save memmory)
# model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
print ("--- %s seconds ---" % round(time.time()-firstTime,4))

unable to import 'smart_open.gcs', disabling that module


--- 39.6364 seconds ---


compares the word of the embedding with the words of our text

In [32]:
# source https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings/comments
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [33]:
oov = check_coverage(vocab,embeddings_index)

100%|█████████████████████████████████████████████████████████████████████████| 60861/60861 [00:01<00:00, 37870.54it/s]


Found embeddings for 39.86% of vocab
Found embeddings for  81.60% of all text


In [34]:
oov[:20]

[('locat', 214788),
 ('friendli', 92033),
 ('servic', 48688),
 ('realli', 44774),
 ('restaur', 38723),
 ('everyth', 36795),
 ('recept', 32698),
 ('amaz', 22735),
 ('coffe', 22678),
 ('comfi', 22325),
 ('arriv', 20525),
 ('expens', 19143),
 ('beauti', 16928),
 ('fantast', 16779),
 ('extrem', 16120),
 ('spaciou', 16102),
 ('welcom', 15916),
 ('noisi', 15077),
 ('upgrad', 14129),
 ('peopl', 13926)]

In [36]:
import string
punctuationList = string.punctuation 
punctuationList = punctuationList+"’" + "”" + "“"+"—"
for punctuation in punctuationList:
    print(punctuation, " in the embedding: ",punctuation in embeddings_index)

!  in the embedding:  True
"  in the embedding:  False
#  in the embedding:  True
$  in the embedding:  True
%  in the embedding:  True
&  in the embedding:  True
'  in the embedding:  True
(  in the embedding:  False
)  in the embedding:  False
*  in the embedding:  True
+  in the embedding:  True
,  in the embedding:  True
-  in the embedding:  True
.  in the embedding:  True
/  in the embedding:  True
:  in the embedding:  True
;  in the embedding:  True
<  in the embedding:  True
=  in the embedding:  True
>  in the embedding:  True
?  in the embedding:  True
@  in the embedding:  True
[  in the embedding:  False
\  in the embedding:  True
]  in the embedding:  False
^  in the embedding:  True
_  in the embedding:  True
`  in the embedding:  True
{  in the embedding:  False
|  in the embedding:  True
}  in the embedding:  False
~  in the embedding:  True
’  in the embedding:  False
”  in the embedding:  False
“  in the embedding:  False
—  in the embedding:  False


In [37]:
for i in range(20):
    print(embeddings_index.index2entity[i])
#TODO: -LRB- and -RRB- are currently used instead of "(" and ")" .... also  ” to " .... also -- has to has a meaning (maybe a placehoder for numbers like in https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings/comments)

the
,
.
of
and
in
a
to
was
''
``
is
for
-rrb-
-lrb-
as
on
with
by
he


In [40]:
ReviewSample = review.sample(n=100, random_state=1) #frac
ScoreSample = score.sample(n=100, random_state=1)

In [41]:
firstTime = time.time()
vectors = []
# for x,y in documents:
#     vectors.append(model.docvecs[y[0]]) 
for text in ReviewSample:
    vectors.append(model.infer_vector(text))
print ("--- %s seconds ---" % round(time.time()-firstTime,4))

--- 122.3167 seconds ---


In [ ]:
vectors[1]

Klassification: Gehört eigentlich in ein anders Notebook, aber faul und soo...

In [44]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    vectors, ScoreSample,test_size=0.2, random_state=42)

print("=======TRAIN=========")
# display(data_train)
# display(target_train)

=======TRAIN=========
